In [ ]:
import math
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout, Activation, Flatten, Convolution1D, MaxPooling1D, Lambda, UpSampling1D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint


def generate_sinusoid(points, frequency, amplitude):
    r = np.zeros(points)
    for x in range(points):
        r[x] = amplitude * math.sin(2 * math.pi * frequency * float(x) / points)
    return r

def add_gaussian_noise(prior, variance):
    for x in range(prior.shape[0]):
        x = x + np.random.normal(0, variance)
    return prior

def add_transient_noise(prior, probability, amplitude):
    for x in range(prior.shape[0]):
        if np.random.uniform() < probability:
            if np.random.uniform() < 0.5:
                x = x - amplitude
            else:
                x = x + amplitude
    return prior
    
def pipeline_generator(batch_size, points):
    while True:
        features = np.zeros((batch_size, points))
        labels = np.zeros((batch_size, points))
        for i in range(batch_size):
            frequency = float(np.random.randint(1, 10))
            amplitude = np.random.uniform(0.05, 1)
            variance = np.random.uniform(0.01, 0.1)
            probability = np.random.uniform(0, 0.0001)
            transient_amplitude = np.random.uniform(0, 0.5)
            signal = generate_sinusoid(points, frequency, amplitude)
            labels[i] = signal
            signal = add_gaussian_noise(signal, variance)
            signal = add_transient_noise(signal, probability, transient_amplitude)
            features[i] = signal
        yield features, labels
    
def encoder_decoder(samples):
    return Sequential([
        Convolution1D(1, 1, activation='sigmoid', input_dim=1, input_length=samples),
        Convolution1D(8, 3, activation='relu', border_mode='same'),
        Convolution1D(8, 3, activation='relu', border_mode='same'),
        MaxPooling1D(2),
        Dropout(0.2),
        Convolution1D(16, 3, activation='relu', border_mode='same'),
        Convolution1D(16, 3, activation='relu', border_mode='same'),
        MaxPooling1D(2),
        Dropout(0.2),
        Convolution1D(32, 3, activation='relu', border_mode='same'),
        Convolution1D(32, 3, activation='relu', border_mode='same'),
        MaxPooling1D(2),
        Dropout(0.2),
        Convolution1D(64, 3, activation='relu', border_mode='same'),
        Convolution1D(64, 3, activation='relu', border_mode='same'),
        MaxPooling1D(2),
        Dropout(0.2),
        Convolution1D(128, 3, activation='relu', border_mode='same'),
        Convolution1D(128, 3, activation='relu', border_mode='same'),
        UpSampling1D(2),
        Dropout(0.2),
        Convolution1D(64, 3, activation='relu', border_mode='same'),
        Convolution1D(64, 3, activation='relu', border_mode='same'),
        UpSampling1D(2),
        Dropout(0.2),
        Convolution1D(32, 3, activation='relu', border_mode='same'),
        Convolution1D(32, 3, activation='relu', border_mode='same'),
        UpSampling1D(2),
        Dropout(0.2),
        Convolution1D(16, 3, activation='relu', border_mode='same'),
        Convolution1D(16, 3, activation='relu', border_mode='same'),
        UpSampling1D(2),
        Dropout(0.2),
        Convolution1D(8, 3, activation='relu', border_mode='same'),
        Convolution1D(8, 3, activation='relu', border_mode='same'),
        Convolution1D(1, 1, activation='sigmoid')
    ])    
    

SAMPLES = 100
BATCH_SIZE = 10
SAMPLES_PER_EPOCH = 200
EPOCHS = 10

model = encoder_decoder(SAMPLES)
generator = pipeline_generator(BATCH_SIZE, SAMPLES)
checkpoint = ModelCheckpoint(filepath='model_weights.h5', verbose=True, save_best_only=True)
optimizer = Adam(lr=1.0e-5)

model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

print('Starting fit...')
model.fit_generator(generator, samples_per_epoch=SAMPLES_PER_EPOCH, nb_epoch=EPOCHS, callbacks=[checkpoint])    
    